In [1]:
import cv2
import numpy as np

In [45]:
win_name = 'scan'
src = cv2.imread('./dataset/KakaoTalk_20240420_113433452_01.jpg') # 이미지 파일 읽기
img = src.copy()


In [46]:
resize = cv2.resize(img, dsize=(0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_AREA)
cv2.imshow("조절", resize)
cv2.waitKey(0) # param 시간동안 키보드 입력 대기 - 0 인경우 무한대기
cv2.destroyAllWindows()

In [61]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 이미지를 흐리게 처리함 (noise 제거를 위해 사용)
edged = cv2.Canny(gray, 75, 250) # edged를 검출하는 함수 (img, minVal, maxVal)
cv2.imshow('gray', edged)  # 윈도우 창에 보여주기 (윈도우창 title, 이미지)
cv2.waitKey(0) # param 시간동안 키보드 입력 대기 - 0 인경우 무한대기
cv2.destroyAllWindows()

In [48]:
# 컨투어 찾기
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
len(contours)

597

In [49]:
contours[0]

array([[[2561, 4023]],

       [[2561, 4024]],

       [[2562, 4025]],

       [[2562, 4026]],

       [[2563, 4026]],

       [[2565, 4024]],

       [[2565, 4023]]], dtype=int32)

In [51]:
draw = img.copy()
cv2.drawContours(draw,  contours, -1, (0, 255, 0))
cv2.imshow("contours", draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

외곽선 관련 함수
- cv2.arcLength() : 외곽선의 길이를 반환
- cv2.contourArea() : 외곽선이 감싸는 영역의 면적을 반환
- cv2.boundingRect() : 주어진 점을 감싸는 최소 크기의 사각형(바운딩 박스)를 반환
- cv2.minEnclosingCircle() : 주어진 점을 감싸는 최소 크기의 원을 반환
- cv2.minAreaRect() : 주어진 점을 감싸는 최소 크기의 회전된 사각형을 반환
- cv2.approxPolyDP() : 외곽선을 근사화(단순화)
- cv2.fitEllipse() : 주어진 점에 적합한 타원을 반환
- cv2.fitLine() : 주어진 점에 적합한 직선을 반환
- cv2.isContourConvex() : 컨벡스인지를 검사
- cv2.convexHull() : 주어진 점으로부터 컨벤스 헐을 반환
- cv2.convexityDefects() : 주어진 점과 컨벡스 헐로부터 컨벡스 디펙트를 반환

참조 사이트
 - https://deep-learning-study.tistory.com/232
 

In [52]:
# 컨투어들 중에 영역 크기 순으로 정렬
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
for c in contours:
    peri = cv2.arcLength(c, True) # 외곽선 길이
    # print(peri)
    verticles = cv2.approxPolyDP(c, 0.02 * peri, True) # 외곽선 근사화
    if len(verticles) == 4 : 
        break
pts = verticles.reshape(4, 2) # 배열을 4 * 2 크기로 조정

In [53]:
print(pts)
print(pts.sum(axis=1))
print(np.diff(pts, axis=1))
print("좌상단", pts[np.argmin(pts.sum(axis=1))])

[[2485  582]
 [ 516  589]
 [ 346 3528]
 [2663 3600]]
[3067 1105 3874 6263]
[[-1903]
 [   73]
 [ 3182]
 [  937]]
좌상단 [516 589]


In [54]:
# 각각의 좌표 찾기
sumXY = pts.sum(axis=1)
diff = np.diff(pts, axis=1)

topLeft = pts[np.argmin(sumXY)]
bottomRight = pts[np.argmax(sumXY)]
topRight = pts[np.argmin(diff)]
bottomLeft = pts[np.argmax(diff)]

np.float32([topLeft, topRight, bottomRight, bottomLeft])

array([[ 516.,  589.],
       [2485.,  582.],
       [2663., 3600.],
       [ 346., 3528.]], dtype=float32)

In [55]:
# 사진을 변환할 때 사용할 서류의 높이
widthTop = abs(topRight[0] - topLeft[0])
widthBottom = abs(bottomRight[0] - bottomLeft[0])
heightRight = abs(topRight[1] - bottomRight[1])
heightLeft = abs(topLeft[1] - bottomLeft[1])
print(widthBottom, widthTop, heightLeft, heightRight)

width = max([widthTop, widthBottom])
height = max([heightRight, heightLeft])

2317 1969 2939 3018


In [56]:
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
pts2 = np.float32([[0,0], [width, 0], [width, height], [0, height]])

In [57]:
matrix = cv2.getPerspectiveTransform(pts1, pts2) # 좌표를 변환하기 위해 사용할 변환행렬
result = cv2.warpPerspective(img, matrix, (width, height)) # 이미지 변환(변환행렬 적용)
cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [58]:
import easyocr

In [59]:
reader = easyocr.Reader(['ko', 'en'], gpu=True)
txt = reader.readtext(result)
print(txt)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[882, 66], [1454, 66], [1454, 115], [882, 115]], 'AeA proofs: manuscript no. wl27crires', 0.6290433014008742), ([[140, 154], [1153, 154], [1153, 216], [140, 216]], 'resolution When observing at high-resolution (R ~100000) the', 0.9450419374010174), ([[1180, 145], [2180, 145], [2180, 212], [1180, 212]], 'high-resolution study were detected to be strongly blue-shifted', 0.7034905818525103), ([[143, 197], [1153, 197], [1153, 257], [143, 257]], 'dynamics of the atmospheric material can be measured through', 0.6548361912684806), ([[1183, 188], [2176, 188], [2176, 252], [1183, 252]], "(by approximately 1Okm $-1) from the planet' s rest frame and", 0.4417425847404617), ([[144, 241], [1074, 241], [1074, 294], [144, 294]], 'Doppler shifts of the the signals caused by their relative', 0.7731315453403398), ([[1080, 255], [1145, 255], [1145, 282], [1080, 282]], 'mo-', 0.950320303440094), ([[1180, 231], [2105, 231], [2105, 293], [1180, 293]], 'the authors discussed the possibly of this signal be

In [60]:
for x in txt :
    print(x[1])

AeA proofs: manuscript no. wl27crires
resolution When observing at high-resolution (R ~100000) the
high-resolution study were detected to be strongly blue-shifted
dynamics of the atmospheric material can be measured through
(by approximately 1Okm $-1) from the planet' s rest frame and
Doppler shifts of the the signals caused by their relative
mo-
the authors discussed the possibly of this signal being only
tions towards the observer (Dang et al. 2018; Ehrenreich et al.
of a broadened velocity signature , with other
of the signal
2020).
this benefit  high wind
from jct strcams
hidden within the noise
and dayside-to-nightside winds
as
well
as temperature differ-
In this work
we used the CRyogenic InfraRed Echelle Spec-
ences can be deduced from observed line
broadening and asym-
trograph (CRIRES+)
in the K band (1972
2452 nm) and in-
metries (Prinoth et al. 2022; Keles 2021). In previous studies,
dependently investigated the atmospheric composition and its
signals from different parts of